<a href="https://colab.research.google.com/github/AALivanova/BI_Stat_2021/blob/ML_hw5/ML_hw5_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
import math
import pandas as pd
import xgboost
import lightgbm
!pip install catboost
import catboost

from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr
from itertools import combinations
from sklearn.base import BaseEstimator
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestClassifier,
                              ExtraTreesClassifier,
                              VotingClassifier)
from sklearn.tree import (DecisionTreeRegressor,
                          DecisionTreeClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

     |████████████████████████████████| 76.1 MB 1.8 MB/s 


In [2]:
plt.rcParams["figure.figsize"] = 12, 9
sns.set_style("whitegrid")
warnings.filterwarnings("ignore")

SEED = 111
random.seed(SEED)
np.random.seed(SEED)

# Задание 3. Строим большой ансамбль
4 балла + 3 дополнительных за эксперименты и максимально высокий скор

В данной задаче вам нужно диагностировать сердечное заболевание у людей по медицинским показателям.

In [3]:
import pandas as pd
!gdown  --id 1VFbDK-Ad-hpf0_GGCBzn4thdn9mkQ-Y- -O heart.csv -q
heart_dataset = pd.read_csv("heart.csv")

In [4]:
heart_dataset

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [5]:
X = heart_dataset.drop("target", axis=1)
y = heart_dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED)
X_train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
178,43,1,0,120,177,0,0,120,1,2.5,1,0,3
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
201,60,1,0,125,258,0,0,141,1,2.8,1,1,3
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3
153,66,0,2,146,278,0,0,152,0,0.0,1,1,2


Обучите разнообразные классификаторы, приведенные ниже, а также ансамбль VotingClassifier из sklearn.ensemble, объединяющий эти классификаторы с помощью жесткого или мякого голосования (параметр voting = 'hard' или 'soft' соответственно). Оцените качество моделей с помощью кросс-валидации на тренировочном наборе, используя функцию cross_val_score и метрику f1. Часть моделей отсюда мы не проходили, о них можно почитать дополнительно, но в принципе для задания не очень важно знать принципы их работы (но, если есть время, то почитайте, там интересно).

In [6]:
dt = DecisionTreeClassifier(random_state=SEED, max_depth=10, min_samples_leaf=10)
rf = RandomForestClassifier(n_estimators=50, random_state=SEED)
etc = ExtraTreesClassifier(random_state=SEED)
knn = KNeighborsClassifier(n_neighbors=5, weights="distance")
svc_lin = SVC(kernel='linear', probability=True, random_state=SEED)
svc_rbf = SVC(kernel='rbf', probability=True, random_state=SEED)
cat = catboost.CatBoostClassifier(verbose=0, random_seed=SEED)
lgbm = lightgbm.LGBMClassifier(random_state=SEED)
lgbm_rf = lightgbm.LGBMClassifier(boosting_type="rf", bagging_freq=1, bagging_fraction=0.7, random_state=SEED)
xgb = xgboost.XGBClassifier(random_state=SEED)
xgb_rf = xgboost.XGBRFClassifier(random_state=SEED)
lr = LogisticRegression(solver='liblinear', max_iter=10000)
nb = GaussianNB()

base_models = [("DT", dt), ("RF", rf), 
               ("ETC", etc), ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("CAT", cat), ("LGBM", lgbm), 
               ("LGBM_RF", lgbm_rf), ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]

In [7]:
voting_hard = VotingClassifier(estimators= base_models, voting='hard')

voting_soft = VotingClassifier(estimators=base_models, voting='soft')

In [8]:
for model in [dt, rf, cat, etc, knn, svc_lin, svc_rbf, xgb, lgbm, xgb_rf, lgbm_rf, lr, nb, voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(f"{model.__class__.__name__}: {scores.mean()}")

DecisionTreeClassifier: 0.797997226792219
RandomForestClassifier: 0.8328751280279528
CatBoostClassifier: 0.8342715174922052
ExtraTreesClassifier: 0.828174603174603
KNeighborsClassifier: 0.6493313763861709
SVC: 0.8403098469098905
SVC: 0.6973119072190279
XGBClassifier: 0.8380318868123746
LGBMClassifier: 0.8129461544095692
XGBRFClassifier: 0.8488204901143478
LGBMClassifier: 0.8101597492003888
LogisticRegression: 0.8500073681108163
GaussianNB: 0.8140676625250128
VotingClassifier: 0.8490813069126322
VotingClassifier: 0.8583484026522002


Вы можете заметить, что ансамбль показывает хорошее, но не лучшее качество предсказания, попробуем его улучшить. Как вы знаете, ансамбли работают лучше, когда модели, входящие в них не скоррелированы друг с другом. Определите корреляцию предсказаний базовых моделей в ансамбле на тестовом наборе данных, и удалите из ансамбля те модели, чьи предсказания будут сильнее коррелировать с остальными. Воспользуйтесь функцией base_model_pair_correlation_for_voting_clf. Спойлер: далеко не факт, что если вы удалите две модели с корреляцией 0.95, то все станет сильно лучше, здесь все будет немного сложнее. Чтобы добиться максимального качества может понадобиться долгий перебор различных комбинаций моделей. Наилучший скор, который мне удалось достичь, это 0.915, но он получен весьма странной комбинацией алгоритмов, а еще и простым перебором всех вариантов)

In [9]:
def base_model_pair_correlation_for_voting_clf(ensemble, X):
    corrs = []
    base_model_names = [f"{est.__class__.__name__}" for est in ensemble.estimators_]
    for (i, est1), (j, est2) in combinations(enumerate(ensemble.estimators_), 2):
        Xi_test = X
        Xj_test = X

        if not isinstance(est1, SVC):
            ypred_t1 = est1.predict_proba(Xi_test)[:, 1]
        else:
            ypred_t1 = est1.decision_function(Xi_test)


        if not isinstance(est2, SVC):
            ypred_t2 = est2.predict_proba(Xi_test)[:, 1]
        else:
            ypred_t2 = est2.decision_function(Xi_test)
        corrs.append((est1, est2, pearsonr(ypred_t1, ypred_t2)[0]))


    return corrs

In [10]:
voting_soft.fit(X_train, y_train)
base_model_pair_correlation_for_voting_clf(voting_soft, X_test)

[(DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  RandomForestClassifier(n_estimators=50, random_state=111),
  0.8134894007824364),
 (DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  ExtraTreesClassifier(random_state=111),
  0.7749544834941163),
 (DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  KNeighborsClassifier(weights='distance'),
  0.31332949662835446),
 (DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  SVC(kernel='linear', probability=True, random_state=111),
  0.715376472596946),
 (DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  SVC(probability=True, random_state=111),
  0.2820135356695378),
 (DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  0.7875690140798687),
 (DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=111),
  LGBMClassifier(random_state=111),
  0.8294070001470021),


как будто бы надо удалить cat и XGB, пробуем

In [11]:
base_models = [("DT", dt), ("RF", rf), 
               ("ETC", etc), ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("LGBM", lgbm), 
               ("LGBM_RF", lgbm_rf),  
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]

voting_hard = VotingClassifier(estimators= base_models, voting='hard')
voting_soft = VotingClassifier(estimators=base_models, voting='soft')

for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(f"{model.__class__.__name__}: {scores.mean()}")

VotingClassifier: 0.8450142450142449
VotingClassifier: 0.8535431660336342


но ничего не поменялось особо, а стало даже хуже, пробуем другие комбинации
Пробую удалять один классификатор из ансамбля

In [ ]:

for i in range(0,13):
  base_models = [("DT", dt), ("RF", rf), 
               ("ETC", etc), ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("CAT", cat), ("LGBM", lgbm), 
               ("LGBM_RF", lgbm_rf), ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
  base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")


0
VotingClassifier: 0.8431938431938434
0
VotingClassifier: 0.8533870490762153
1
VotingClassifier: 0.851356032357316
1
VotingClassifier: 0.8583484026522002
2
VotingClassifier: 0.8440863019176272
2
VotingClassifier: 0.863016936732499
3
VotingClassifier: 0.8527443105756358
3
VotingClassifier: 0.8551737994775969
4
VotingClassifier: 0.8431938431938434
4
VotingClassifier: 0.8534798534798534
5
VotingClassifier: 0.8527443105756358
5
VotingClassifier: 0.8583484026522002
6
VotingClassifier: 0.8481888481888481
6
VotingClassifier: 0.8583484026522002
7
VotingClassifier: 0.8527055600226333
7
VotingClassifier: 0.84856642687968
8
VotingClassifier: 0.8395804737268152
8
VotingClassifier: 0.8583484026522002
9
VotingClassifier: 0.8518518518518517
9
VotingClassifier: 0.8535431660336342
10
VotingClassifier: 0.8468393205235311
10
VotingClassifier: 0.853949856084979
11
VotingClassifier: 0.8432259510565032
11
VotingClassifier: 0.8583484026522002
12
VotingClassifier: 0.8527443105756358
12
VotingClassifier: 0.85

как будто бы эффективно удалять ETC, удалим и попробуем снова поудалять еще один случайный классификатор

In [ ]:
for i in range(0,12):
  base_models = [("DT", dt), ("RF", rf), 
                ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("CAT", cat), ("LGBM", lgbm), 
               ("LGBM_RF", lgbm_rf), ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
  base_models_pop = base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")

0
VotingClassifier: 0.848754835997926
0
VotingClassifier: 0.8541613425334357
1
VotingClassifier: 0.863016936732499
1
VotingClassifier: 0.863016936732499
2
VotingClassifier: 0.8550958684827493
2
VotingClassifier: 0.8583484026522002
3
VotingClassifier: 0.848754835997926
3
VotingClassifier: 0.863016936732499
4
VotingClassifier: 0.853949856084979
4
VotingClassifier: 0.8583484026522002
5
VotingClassifier: 0.8630952380952381
5
VotingClassifier: 0.863016936732499
6
VotingClassifier: 0.8499095840867993
6
VotingClassifier: 0.8464285714285715
7
VotingClassifier: 0.854578118167098
7
VotingClassifier: 0.863016936732499
8
VotingClassifier: 0.8490813069126322
8
VotingClassifier: 0.8547619047619047
9
VotingClassifier: 0.854578118167098
9
VotingClassifier: 0.863016936732499
10
VotingClassifier: 0.8499095840867993
10
VotingClassifier: 0.8547619047619047
11
VotingClassifier: 0.8597644025630481
11
VotingClassifier: 0.8594304388422035


как будто бы полезно удалить классификатор с индексом 1

In [ ]:
for i in range(0,11):
  base_models = [("DT", dt),  
                ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("CAT", cat), ("LGBM", lgbm), 
               ("LGBM_RF", lgbm_rf), ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
  base_models_pop = base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")

0
VotingClassifier: 0.8514242958687402
0
VotingClassifier: 0.8591380816873899
1
VotingClassifier: 0.856368563685637
1
VotingClassifier: 0.8551737994775969
2
VotingClassifier: 0.8425841025327546
2
VotingClassifier: 0.863016936732499
3
VotingClassifier: 0.856368563685637
3
VotingClassifier: 0.8583484026522002
4
VotingClassifier: 0.861981984599035
4
VotingClassifier: 0.8675769002527908
5
VotingClassifier: 0.8558406363284412
5
VotingClassifier: 0.8535431660336342
6
VotingClassifier: 0.8531533825651474
6
VotingClassifier: 0.8675769002527908
7
VotingClassifier: 0.8502887443210136
7
VotingClassifier: 0.8583954867087398
8
VotingClassifier: 0.8594861136581292
8
VotingClassifier: 0.863016936732499
9
VotingClassifier: 0.8473492969371219
9
VotingClassifier: 0.863016936732499
10
VotingClassifier: 0.8527443105756358
10
VotingClassifier: 0.8594304388422035


удалим 4

In [ ]:
for i in range(0,10):
  base_models = [("DT", dt),  
                ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("LGBM", lgbm), 
               ("LGBM_RF", lgbm_rf), ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
  base_models_pop = base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")

0
VotingClassifier: 0.8634585442151296
0
VotingClassifier: 0.8547395351201686
1
VotingClassifier: 0.8599206349206349
1
VotingClassifier: 0.8515873015873016
2
VotingClassifier: 0.8666505186793337
2
VotingClassifier: 0.863016936732499
3
VotingClassifier: 0.8599206349206349
3
VotingClassifier: 0.8598249622682946
4
VotingClassifier: 0.8604534416736715
4
VotingClassifier: 0.851538712991501
5
VotingClassifier: 0.8598249622682946
5
VotingClassifier: 0.8675769002527908
6
VotingClassifier: 0.8604534416736715
6
VotingClassifier: 0.8630640207890385
7
VotingClassifier: 0.8634585442151296
7
VotingClassifier: 0.8631783536855696
8
VotingClassifier: 0.8551737994775969
8
VotingClassifier: 0.863016936732499
9
VotingClassifier: 0.8677637721755369
9
VotingClassifier: 0.8594304388422035


удалим 5

In [ ]:
for i in range(0,9):
  base_models = [("DT", dt),  
                ("KNN", knn), 
               ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
               ("LGBM", lgbm), 
                ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
  base_models_pop = base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")

0
VotingClassifier: 0.8491871146437228
0
VotingClassifier: 0.8595918557952742
1
VotingClassifier: 0.8484848484848486
1
VotingClassifier: 0.8598249622682946
2
VotingClassifier: 0.8560245664376148
2
VotingClassifier: 0.8631783536855696
3
VotingClassifier: 0.8571428571428571
3
VotingClassifier: 0.8675769002527908
4
VotingClassifier: 0.8539388322520852
4
VotingClassifier: 0.854578118167098
5
VotingClassifier: 0.844535951853025
5
VotingClassifier: 0.8639904023624955
6
VotingClassifier: 0.8476863931430012
6
VotingClassifier: 0.8631783536855696
7
VotingClassifier: 0.8527443105756358
7
VotingClassifier: 0.8631783536855696
8
VotingClassifier: 0.8618113912231559
8
VotingClassifier: 0.8594304388422035


In [ ]:
for i in range(0,8):
  base_models = [("DT", dt),  
                ("KNN", knn), 
               ("SVC_LIN", svc_lin),  
               ("LGBM", lgbm), 
                ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
  base_models_pop = base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")

0
VotingClassifier: 0.8595006526041008
0
VotingClassifier: 0.8547395351201686
1
VotingClassifier: 0.8551737994775969
1
VotingClassifier: 0.8551737994775969
2
VotingClassifier: 0.8666953961071607
2
VotingClassifier: 0.8675769002527908
3
VotingClassifier: 0.8614532019704434
3
VotingClassifier: 0.8518399178107781
4
VotingClassifier: 0.8567063665274054
4
VotingClassifier: 0.8518399178107781
5
VotingClassifier: 0.8631636562671045
5
VotingClassifier: 0.8631783536855696
6
VotingClassifier: 0.8588522588522588
6
VotingClassifier: 0.8598249622682946
7
VotingClassifier: 0.8714557552978999
7
VotingClassifier: 0.8594304388422035


In [ ]:
for i in range(0,6):
  base_models = [("DT", dt),  
                ("KNN", knn), 
  
                
                ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr)]
  base_models_pop = base_models.pop(i)
  voting_hard = VotingClassifier(estimators= base_models, voting='hard')
  voting_soft = VotingClassifier(estimators=base_models, voting='soft')

  for model in [voting_hard, voting_soft]: 
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
    print(i)
    print(f"{model.__class__.__name__}: {scores.mean()}")

0
VotingClassifier: 0.8691160372194854
0
VotingClassifier: 0.8639904023624955
1
VotingClassifier: 0.8594944150499707
1
VotingClassifier: 0.8451285552093847
2
VotingClassifier: 0.874723729154109
2
VotingClassifier: 0.8455110375195781
3
VotingClassifier: 0.8614532019704434
3
VotingClassifier: 0.852740521991168
4
VotingClassifier: 0.8691160372194854
4
VotingClassifier: 0.8607827038861521
5
VotingClassifier: 0.8630952380952381
5
VotingClassifier: 0.8455110375195781


## Короче говоря, таким нехитрым образом я перебирала варианты, пока не нашла лучшее сочетание

In [12]:
base_models = [("DT", dt),  
                ("KNN", knn), 
  
                
                ("XGB", xgb), 
               ("XGB_RF", xgb_rf), ("LR", lr)]

voting_hard = VotingClassifier(estimators= base_models, voting='hard')
voting_soft = VotingClassifier(estimators=base_models, voting='soft')

for model in [voting_hard, voting_soft]: 
  scores = cross_val_score(model, X_train, y_train, cv=3, scoring="f1")
  print(f"{model.__class__.__name__}: {scores.mean()}")

VotingClassifier: 0.874723729154109
VotingClassifier: 0.8455110375195781


Можно посчитать f1 скор на тесте таким сочетанием классификаторов в ансамбле

In [34]:
from sklearn.metrics import f1_score
voting_hard.fit(X_train, y_train)

y_pred = voting_hard.predict(X_test)
f1_score(y_test, y_pred)

0.8910891089108911